## Import Libraries

In [62]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import squareform, pdist,jaccard
import sys,  datetime, os
import requests

In [63]:
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import MACCSkeys
from rdkit.Chem import PandasTools
from rdkit import RDConfig
from rdkit.Chem import AllChem
from rdkit import Avalon
from rdkit.Chem.AtomPairs import Pairs
from rdkit.Chem.Fingerprints import ClusterMols
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.Pharm2D import Gobbi_Pharm2D,Generate
from rdkit.Chem.Pharm2D.SigFactory import SigFactory
from rdkit.Chem import ChemicalFeatures

## Load Data: Training

In [75]:
df = pd.read_table('Output/PubChemID_SMILES_InchI_pertid_2018_08.tsv')
df.head()

,pert_id,InChIKeys,SMILEs,Unnamed: 0
0,BRD-A60245366,RCYPVQCPYKNSTG-UHFFFAOYSA-N,C1=CC=C2C(=C1)N=C(S2)C(C#N)C3=NC(=NC=C3)NCCC4=CN=CC=C4,3.0
1,BRD-K18135438,RUDATBOHQWOJDD-BSWAIDMHSA-N,CC(CCC(=O)O)C1CCC2C1(CCC3C2C(CC4C3(CCC(C4)O)C)O)C,5.0
2,BRD-K54771420,RFDAIACWWDREDC-FRVQLJSFSA-N,CC(CCC(=O)NCC(=O)O)C1CCC2C1(C(CC3C2C(CC4C3(CCC(C4)O)C)O)O)C,6.0
3,BRD-K88573743,YWTBGJGMTBHQTM-IBGZPJMESA-N,CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CNC5=CC=CC=C54)N,10.0
4,BRD-K66175015,ULXXDDBFHOBEHA-CWDCEQMOSA-N,CN(C)CC=CC(=O)NC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC(=C(C=C3)F)Cl)OC4CCOC4,12.0


In [76]:
df = df.drop(columns = ['InChIKeys', 'Unnamed: 0'])

In [77]:
df.head(2)

,pert_id,SMILEs
0,BRD-A60245366,C1=CC=C2C(=C1)N=C(S2)C(C#N)C3=NC(=NC=C3)NCCC4=CN=CC=C4
1,BRD-K18135438,CC(CCC(=O)O)C1CCC2C1(CCC3C2C(CC4C3(CCC(C4)O)C)O)C


## Molecule Column

In [78]:
molecule = [Chem.MolFromSmiles(x) for x in df['SMILEs']]
df.loc[:,'Molecule'] = pd.Series(molecule, index=df.index)

In [79]:
#Check that there is no null values in the df
None in df['Molecule'], len(df['Molecule'])

(False, 5435)

In [80]:
all_drugs = list(df["pert_id"])

## MACCs Fingerprints

In [81]:
macc_fps = [MACCSkeys.GenMACCSKeys(x) for x in df['Molecule']]

In [83]:
macc_np_fps = []
for fp in macc_fps:
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp, arr)
    macc_np_fps.append(arr)

macc_df = pd.DataFrame(macc_np_fps, index = all_drugs)

In [84]:
column_labels = []
for col in macc_df.columns:
    column_labels.append('MACC_'+ str(col))

macc_df.columns = column_labels

In [85]:
macc_df.head(2)

,MACC_0,MACC_1,MACC_2,MACC_3,MACC_4,MACC_5,MACC_6,MACC_7,MACC_8,MACC_9,MACC_10,MACC_11,MACC_12,MACC_13,MACC_14,MACC_15,MACC_16,MACC_17,MACC_18,MACC_19,MACC_20,MACC_21,MACC_22,MACC_23,MACC_24,MACC_25,MACC_26,MACC_27,MACC_28,MACC_29,MACC_30,MACC_31,MACC_32,MACC_33,MACC_34,MACC_35,MACC_36,MACC_37,MACC_38,MACC_39,MACC_40,MACC_41,MACC_42,MACC_43,MACC_44,MACC_45,MACC_46,MACC_47,MACC_48,MACC_49,MACC_50,MACC_51,MACC_52,MACC_53,MACC_54,MACC_55,MACC_56,MACC_57,MACC_58,MACC_59,MACC_60,MACC_61,MACC_62,MACC_63,MACC_64,MACC_65,MACC_66,MACC_67,MACC_68,MACC_69,MACC_70,MACC_71,MACC_72,MACC_73,MACC_74,MACC_75,MACC_76,MACC_77,MACC_78,MACC_79,MACC_80,MACC_81,MACC_82,MACC_83,MACC_84,MACC_85,MACC_86,MACC_87,MACC_88,MACC_89,MACC_90,MACC_91,MACC_92,MACC_93,MACC_94,MACC_95,MACC_96,MACC_97,MACC_98,MACC_99,MACC_100,MACC_101,MACC_102,MACC_103,MACC_104,MACC_105,MACC_106,MACC_107,MACC_108,MACC_109,MACC_110,MACC_111,MACC_112,MACC_113,MACC_114,MACC_115,MACC_116,MACC_117,MACC_118,MACC_119,MACC_120,MACC_121,MACC_122,MACC_123,MACC_124,MACC_125,MACC_126,MACC_127,MACC_128,MACC_129,MACC_130,MACC_131,MACC_132,MACC_133,MACC_134,MACC_135,MACC_136,MACC_137,MACC_138,MACC_139,MACC_140,MACC_141,MACC_142,MACC_143,MACC_144,MACC_145,MACC_146,MACC_147,MACC_148,MACC_149,MACC_150,MACC_151,MACC_152,MACC_153,MACC_154,MACC_155,MACC_156,MACC_157,MACC_158,MACC_159,MACC_160,MACC_161,MACC_162,MACC_163,MACC_164,MACC_165,MACC_166
BRD-A60245366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0
BRD-K18135438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0


In [86]:
filename = 'Output/MACCs_BMatrix_map_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
macc_df.to_csv(filename, sep='\t', compression='gzip')

## Morgan Fingerprints

In [90]:
#change radius and useFeatures = True as needed 
morg_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 4) for x in df['Molecule']]

morg_np_fps = []
for fp in morg_fps:
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp, arr)
    morg_np_fps.append(arr)

morg_df = pd.DataFrame(morg_np_fps, index = all_drugs)

column_labels = []
for col in morg_df.columns:
    column_labels.append('Morg4_' + str(col))
morg_df.columns = column_labels

filename = 'Output/Morg4_BMatrix_map_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
morg_df.to_csv(filename, sep='\t', compression='gzip')

## Avalon Fingerprints

In [91]:
avalon_fps = [Avalon.pyAvalonTools.GetAvalonFP(x) for x in df['Molecule']]

In [92]:
avalon_np_fps = []
for fp in avalon_fps:
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp, arr)
    avalon_np_fps.append(arr)

avalon_df = pd.DataFrame(avalon_np_fps, index = all_drugs)

In [93]:
column_labels = []
for col in avalon_df.columns:
    column_labels.append('Avalon_' + str(col))
avalon_df.columns = column_labels

In [94]:
avalon_df.head(2)

,Avalon_0,Avalon_1,Avalon_2,Avalon_3,Avalon_4,Avalon_5,Avalon_6,Avalon_7,Avalon_8,Avalon_9,Avalon_10,Avalon_11,Avalon_12,Avalon_13,Avalon_14,Avalon_15,Avalon_16,Avalon_17,Avalon_18,Avalon_19,Avalon_20,Avalon_21,Avalon_22,Avalon_23,Avalon_24,Avalon_25,Avalon_26,Avalon_27,Avalon_28,Avalon_29,Avalon_30,Avalon_31,Avalon_32,Avalon_33,Avalon_34,Avalon_35,Avalon_36,Avalon_37,Avalon_38,Avalon_39,Avalon_40,Avalon_41,Avalon_42,Avalon_43,Avalon_44,Avalon_45,Avalon_46,Avalon_47,Avalon_48,Avalon_49,Avalon_50,Avalon_51,Avalon_52,Avalon_53,Avalon_54,Avalon_55,Avalon_56,Avalon_57,Avalon_58,Avalon_59,Avalon_60,Avalon_61,Avalon_62,Avalon_63,Avalon_64,Avalon_65,Avalon_66,Avalon_67,Avalon_68,Avalon_69,Avalon_70,Avalon_71,Avalon_72,Avalon_73,Avalon_74,Avalon_75,Avalon_76,Avalon_77,Avalon_78,Avalon_79,Avalon_80,Avalon_81,Avalon_82,Avalon_83,Avalon_84,Avalon_85,Avalon_86,Avalon_87,Avalon_88,Avalon_89,Avalon_90,Avalon_91,Avalon_92,Avalon_93,Avalon_94,Avalon_95,Avalon_96,Avalon_97,Avalon_98,Avalon_99,Avalon_100,Avalon_101,Avalon_102,Avalon_103,Avalon_104,Avalon_105,Avalon_106,Avalon_107,Avalon_108,Avalon_109,Avalon_110,Avalon_111,Avalon_112,Avalon_113,Avalon_114,Avalon_115,Avalon_116,Avalon_117,Avalon_118,Avalon_119,Avalon_120,Avalon_121,Avalon_122,Avalon_123,Avalon_124,Avalon_125,Avalon_126,Avalon_127,Avalon_128,Avalon_129,Avalon_130,Avalon_131,Avalon_132,Avalon_133,Avalon_134,Avalon_135,Avalon_136,Avalon_137,Avalon_138,Avalon_139,Avalon_140,Avalon_141,Avalon_142,Avalon_143,Avalon_144,Avalon_145,Avalon_146,Avalon_147,Avalon_148,Avalon_149,Avalon_150,Avalon_151,Avalon_152,Avalon_153,Avalon_154,Avalon_155,Avalon_156,Avalon_157,Avalon_158,Avalon_159,Avalon_160,Avalon_161,Avalon_162,Avalon_163,Avalon_164,Avalon_165,Avalon_166,Avalon_167,Avalon_168,Avalon_169,Avalon_170,Avalon_171,Avalon_172,Avalon_173,Avalon_174,Avalon_175,Avalon_176,Avalon_177,Avalon_178,Avalon_179,Avalon_180,Avalon_181,Avalon_182,Avalon_183,Avalon_184,Avalon_185,Avalon_186,Avalon_187,Avalon_188,Avalon_189,Avalon_190,Avalon_191,Avalon_192,Avalon_193,Avalon_194,Avalon_195,Avalon_196,Avalon_197,Avalon_198,Avalon_199,Avalon_200,Avalon_201,Avalon_202,Avalon_203,Avalon_204,Avalon_205,Avalon_206,Avalon_207,Avalon_208,Avalon_209,Avalon_210,Avalon_211,Avalon_212,Avalon_213,Avalon_214,Avalon_215,Avalon_216,Avalon_217,Avalon_218,Avalon_219,Avalon_220,Avalon_221,Avalon_222,Avalon_223,Avalon_224,Avalon_225,Avalon_226,Avalon_227,Avalon_228,Avalon_229,Avalon_230,Avalon_231,Avalon_232,Avalon_233,Avalon_234,Avalon_235,Avalon_236,Avalon_237,Avalon_238,Avalon_239,Avalon_240,Avalon_241,Avalon_242,Avalon_243,Avalon_244,Avalon_245,Avalon_246,Avalon_247,Avalon_248,Avalon_249,Avalon_250,Avalon_251,Avalon_252,Avalon_253,Avalon_254,Avalon_255,Avalon_256,Avalon_257,Avalon_258,Avalon_259,Avalon_260,Avalon_261,Avalon_262,Avalon_263,Avalon_264,Avalon_265,Avalon_266,Avalon_267,Avalon_268,Avalon_269,Avalon_270,Avalon_271,Avalon_272,Avalon_273,Avalon_274,Avalon_275,Avalon_276,Avalon_277,Avalon_278,Avalon_279,Avalon_280,Avalon_281,Avalon_282,Avalon_283,Avalon_284,Avalon_285,Avalon_286,Avalon_287,Avalon_288,Avalon_289,Avalon_290,Avalon_291,Avalon_292,Avalon_293,Avalon_294,Avalon_295,Avalon_296,Avalon_297,Avalon_298,Avalon_299,Avalon_300,Avalon_301,Avalon_302,Avalon_303,Avalon_304,Avalon_305,Avalon_306,Avalon_307,Avalon_308,Avalon_309,Avalon_310,Avalon_311,Avalon_312,Avalon_313,Avalon_314,Avalon_315,Avalon_316,Avalon_317,Avalon_318,Avalon_319,Avalon_320,Avalon_321,Avalon_322,Avalon_323,Avalon_324,Avalon_325,Avalon_326,Avalon_327,Avalon_328,Avalon_329,Avalon_330,Avalon_331,Avalon_332,Avalon_333,Avalon_334,Avalon_335,Avalon_336,Avalon_337,Avalon_338,Avalon_339,Avalon_340,Avalon_341,Avalon_342,Avalon_343,Avalon_344,Avalon_345,Avalon_346,Avalon_347,Avalon_348,Avalon_349,Avalon_350,Avalon_351,Avalon_352,Avalon_353,Avalon_354,Avalon_355,Avalon_356,Avalon_357,Avalon_358,Avalon_359,Avalon_360,Avalon_361,Avalon_362,Avalon_363,Avalon_364,Avalon_365,Avalon_366,Avalon_367,Avalon_368,Avalon_369,Avalon_370,Avalon_371,Avalon_372,Avalon

In [95]:
filename = 'Output/Avalon_BMatrix_map_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
avalon_df.to_csv(filename, sep='\t', compression='gzip')

## Atom Pairs Fingerprints

In [96]:
atom_pairs_fps = [rdMolDescriptors.GetHashedAtomPairFingerprintAsBitVect(x) for x in df['Molecule']]

In [98]:
ap_np_fps = []
for fp in atom_pairs_fps:
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp, arr)
    ap_np_fps.append(arr)

ap_df = pd.DataFrame(ap_np_fps, index = all_drugs)

In [99]:
column_labels = []
for col in ap_df.columns:
    column_labels.append('AtomPair_' + str(col))
ap_df.columns = column_labels

In [100]:
ap_df.head(2)

,AtomPair_0,AtomPair_1,AtomPair_2,AtomPair_3,AtomPair_4,AtomPair_5,AtomPair_6,AtomPair_7,AtomPair_8,AtomPair_9,AtomPair_10,AtomPair_11,AtomPair_12,AtomPair_13,AtomPair_14,AtomPair_15,AtomPair_16,AtomPair_17,AtomPair_18,AtomPair_19,AtomPair_20,AtomPair_21,AtomPair_22,AtomPair_23,AtomPair_24,AtomPair_25,AtomPair_26,AtomPair_27,AtomPair_28,AtomPair_29,AtomPair_30,AtomPair_31,AtomPair_32,AtomPair_33,AtomPair_34,AtomPair_35,AtomPair_36,AtomPair_37,AtomPair_38,AtomPair_39,AtomPair_40,AtomPair_41,AtomPair_42,AtomPair_43,AtomPair_44,AtomPair_45,AtomPair_46,AtomPair_47,AtomPair_48,AtomPair_49,AtomPair_50,AtomPair_51,AtomPair_52,AtomPair_53,AtomPair_54,AtomPair_55,AtomPair_56,AtomPair_57,AtomPair_58,AtomPair_59,AtomPair_60,AtomPair_61,AtomPair_62,AtomPair_63,AtomPair_64,AtomPair_65,AtomPair_66,AtomPair_67,AtomPair_68,AtomPair_69,AtomPair_70,AtomPair_71,AtomPair_72,AtomPair_73,AtomPair_74,AtomPair_75,AtomPair_76,AtomPair_77,AtomPair_78,AtomPair_79,AtomPair_80,AtomPair_81,AtomPair_82,AtomPair_83,AtomPair_84,AtomPair_85,AtomPair_86,AtomPair_87,AtomPair_88,AtomPair_89,AtomPair_90,AtomPair_91,AtomPair_92,AtomPair_93,AtomPair_94,AtomPair_95,AtomPair_96,AtomPair_97,AtomPair_98,AtomPair_99,AtomPair_100,AtomPair_101,AtomPair_102,AtomPair_103,AtomPair_104,AtomPair_105,AtomPair_106,AtomPair_107,AtomPair_108,AtomPair_109,AtomPair_110,AtomPair_111,AtomPair_112,AtomPair_113,AtomPair_114,AtomPair_115,AtomPair_116,AtomPair_117,AtomPair_118,AtomPair_119,AtomPair_120,AtomPair_121,AtomPair_122,AtomPair_123,AtomPair_124,AtomPair_125,AtomPair_126,AtomPair_127,AtomPair_128,AtomPair_129,AtomPair_130,AtomPair_131,AtomPair_132,AtomPair_133,AtomPair_134,AtomPair_135,AtomPair_136,AtomPair_137,AtomPair_138,AtomPair_139,AtomPair_140,AtomPair_141,AtomPair_142,AtomPair_143,AtomPair_144,AtomPair_145,AtomPair_146,AtomPair_147,AtomPair_148,AtomPair_149,AtomPair_150,AtomPair_151,AtomPair_152,AtomPair_153,AtomPair_154,AtomPair_155,AtomPair_156,AtomPair_157,AtomPair_158,AtomPair_159,AtomPair_160,AtomPair_161,AtomPair_162,AtomPair_163,AtomPair_164,AtomPair_165,AtomPair_166,AtomPair_167,AtomPair_168,AtomPair_169,AtomPair_170,AtomPair_171,AtomPair_172,AtomPair_173,AtomPair_174,AtomPair_175,AtomPair_176,AtomPair_177,AtomPair_178,AtomPair_179,AtomPair_180,AtomPair_181,AtomPair_182,AtomPair_183,AtomPair_184,AtomPair_185,AtomPair_186,AtomPair_187,AtomPair_188,AtomPair_189,AtomPair_190,AtomPair_191,AtomPair_192,AtomPair_193,AtomPair_194,AtomPair_195,AtomPair_196,AtomPair_197,AtomPair_198,AtomPair_199,AtomPair_200,AtomPair_201,AtomPair_202,AtomPair_203,AtomPair_204,AtomPair_205,AtomPair_206,AtomPair_207,AtomPair_208,AtomPair_209,AtomPair_210,AtomPair_211,AtomPair_212,AtomPair_213,AtomPair_214,AtomPair_215,AtomPair_216,AtomPair_217,AtomPair_218,AtomPair_219,AtomPair_220,AtomPair_221,AtomPair_222,AtomPair_223,AtomPair_224,AtomPair_225,AtomPair_226,AtomPair_227,AtomPair_228,AtomPair_229,AtomPair_230,AtomPair_231,AtomPair_232,AtomPair_233,AtomPair_234,AtomPair_235,AtomPair_236,AtomPair_237,AtomPair_238,AtomPair_239,AtomPair_240,AtomPair_241,AtomPair_242,AtomPair_243,AtomPair_244,AtomPair_245,AtomPair_246,AtomPair_247,AtomPair_248,AtomPair_249,AtomPair_250,AtomPair_251,AtomPair_252,AtomPair_253,AtomPair_254,AtomPair_255,AtomPair_256,AtomPair_257,AtomPair_258,AtomPair_259,AtomPair_260,AtomPair_261,AtomPair_262,AtomPair_263,AtomPair_264,AtomPair_265,AtomPair_266,AtomPair_267,AtomPair_268,AtomPair_269,AtomPair_270,AtomPair_271,AtomPair_272,AtomPair_273,AtomPair_274,AtomPair_275,AtomPair_276,AtomPair_277,AtomPair_278,AtomPair_279,AtomPair_280,AtomPair_281,AtomPair_282,AtomPair_283,AtomPair_284,AtomPair_285,AtomPair_286,AtomPair_287,AtomPair_288,AtomPair_289,AtomPair_290,AtomPair_291,AtomPair_292,AtomPair_293,AtomPair_294,AtomPair_295,AtomPair_296,AtomPair_297,AtomPair_298,AtomPair_299,AtomPair_300,AtomPair_301,AtomPair_302,AtomPair_303,AtomPair_304,AtomPair_305,AtomPair_306,AtomPair_307,AtomPair_308,AtomPair_309,AtomPair_310,AtomPair_311,AtomPair_312,AtomPair_313,AtomPair_314,AtomPair_315,A

In [101]:
filename = 'Output/AtomPair_BMatrix_map_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
ap_df.to_csv(filename, sep='\t', compression='gzip')

## Topological Fingerprints

In [102]:
top_fps = [FingerprintMols.FingerprintMol(x , minPath = 1, maxPath = 7, fpSize = 2048, bitsPerHash = 2, useHs = True, tgtDensity = 0, minSize = 128) for x in df['Molecule']]

In [103]:
top_np_fps = []
for fp in top_fps:
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp, arr)
    top_np_fps.append(arr)

top_df = pd.DataFrame(top_np_fps, index = all_drugs)

In [104]:
column_labels = []
for col in top_df.columns:
    column_labels.append('Top_' + str(col))
top_df.columns = column_labels

In [105]:
top_df.head(2)

,Top_0,Top_1,Top_2,Top_3,Top_4,Top_5,Top_6,Top_7,Top_8,Top_9,Top_10,Top_11,Top_12,Top_13,Top_14,Top_15,Top_16,Top_17,Top_18,Top_19,Top_20,Top_21,Top_22,Top_23,Top_24,Top_25,Top_26,Top_27,Top_28,Top_29,Top_30,Top_31,Top_32,Top_33,Top_34,Top_35,Top_36,Top_37,Top_38,Top_39,Top_40,Top_41,Top_42,Top_43,Top_44,Top_45,Top_46,Top_47,Top_48,Top_49,Top_50,Top_51,Top_52,Top_53,Top_54,Top_55,Top_56,Top_57,Top_58,Top_59,Top_60,Top_61,Top_62,Top_63,Top_64,Top_65,Top_66,Top_67,Top_68,Top_69,Top_70,Top_71,Top_72,Top_73,Top_74,Top_75,Top_76,Top_77,Top_78,Top_79,Top_80,Top_81,Top_82,Top_83,Top_84,Top_85,Top_86,Top_87,Top_88,Top_89,Top_90,Top_91,Top_92,Top_93,Top_94,Top_95,Top_96,Top_97,Top_98,Top_99,Top_100,Top_101,Top_102,Top_103,Top_104,Top_105,Top_106,Top_107,Top_108,Top_109,Top_110,Top_111,Top_112,Top_113,Top_114,Top_115,Top_116,Top_117,Top_118,Top_119,Top_120,Top_121,Top_122,Top_123,Top_124,Top_125,Top_126,Top_127,Top_128,Top_129,Top_130,Top_131,Top_132,Top_133,Top_134,Top_135,Top_136,Top_137,Top_138,Top_139,Top_140,Top_141,Top_142,Top_143,Top_144,Top_145,Top_146,Top_147,Top_148,Top_149,Top_150,Top_151,Top_152,Top_153,Top_154,Top_155,Top_156,Top_157,Top_158,Top_159,Top_160,Top_161,Top_162,Top_163,Top_164,Top_165,Top_166,Top_167,Top_168,Top_169,Top_170,Top_171,Top_172,Top_173,Top_174,Top_175,Top_176,Top_177,Top_178,Top_179,Top_180,Top_181,Top_182,Top_183,Top_184,Top_185,Top_186,Top_187,Top_188,Top_189,Top_190,Top_191,Top_192,Top_193,Top_194,Top_195,Top_196,Top_197,Top_198,Top_199,Top_200,Top_201,Top_202,Top_203,Top_204,Top_205,Top_206,Top_207,Top_208,Top_209,Top_210,Top_211,Top_212,Top_213,Top_214,Top_215,Top_216,Top_217,Top_218,Top_219,Top_220,Top_221,Top_222,Top_223,Top_224,Top_225,Top_226,Top_227,Top_228,Top_229,Top_230,Top_231,Top_232,Top_233,Top_234,Top_235,Top_236,Top_237,Top_238,Top_239,Top_240,Top_241,Top_242,Top_243,Top_244,Top_245,Top_246,Top_247,Top_248,Top_249,Top_250,Top_251,Top_252,Top_253,Top_254,Top_255,Top_256,Top_257,Top_258,Top_259,Top_260,Top_261,Top_262,Top_263,Top_264,Top_265,Top_266,Top_267,Top_268,Top_269,Top_270,Top_271,Top_272,Top_273,Top_274,Top_275,Top_276,Top_277,Top_278,Top_279,Top_280,Top_281,Top_282,Top_283,Top_284,Top_285,Top_286,Top_287,Top_288,Top_289,Top_290,Top_291,Top_292,Top_293,Top_294,Top_295,Top_296,Top_297,Top_298,Top_299,Top_300,Top_301,Top_302,Top_303,Top_304,Top_305,Top_306,Top_307,Top_308,Top_309,Top_310,Top_311,Top_312,Top_313,Top_314,Top_315,Top_316,Top_317,Top_318,Top_319,Top_320,Top_321,Top_322,Top_323,Top_324,Top_325,Top_326,Top_327,Top_328,Top_329,Top_330,Top_331,Top_332,Top_333,Top_334,Top_335,Top_336,Top_337,Top_338,Top_339,Top_340,Top_341,Top_342,Top_343,Top_344,Top_345,Top_346,Top_347,Top_348,Top_349,Top_350,Top_351,Top_352,Top_353,Top_354,Top_355,Top_356,Top_357,Top_358,Top_359,Top_360,Top_361,Top_362,Top_363,Top_364,Top_365,Top_366,Top_367,Top_368,Top_369,Top_370,Top_371,Top_372,Top_373,Top_374,Top_375,Top_376,Top_377,Top_378,Top_379,Top_380,Top_381,Top_382,Top_383,Top_384,Top_385,Top_386,Top_387,Top_388,Top_389,Top_390,Top_391,Top_392,Top_393,Top_394,Top_395,Top_396,Top_397,Top_398,Top_399,Top_400,Top_401,Top_402,Top_403,Top_404,Top_405,Top_406,Top_407,Top_408,Top_409,Top_410,Top_411,Top_412,Top_413,Top_414,Top_415,Top_416,Top_417,Top_418,Top_419,Top_420,Top_421,Top_422,Top_423,Top_424,Top_425,Top_426,Top_427,Top_428,Top_429,Top_430,Top_431,Top_432,Top_433,Top_434,Top_435,Top_436,Top_437,Top_438,Top_439,Top_440,Top_441,Top_442,Top_443,Top_444,Top_445,Top_446,Top_447,Top_448,Top_449,Top_450,Top_451,Top_452,Top_453,Top_454,Top_455,Top_456,Top_457,Top_458,Top_459,Top_460,Top_461,Top_462,Top_463,Top_464,Top_465,Top_466,Top_467,Top_468,Top_469,Top_470,Top_471,Top_472,Top_473,Top_474,Top_475,Top_476,Top_477,Top_478,Top_479,Top_480,Top_481,Top_482,Top_483,Top_484,Top_485,Top_486,Top_487,Top_488,Top_489,Top_490,Top_491,Top_492,Top_493,Top_494,Top_495,Top_496,Top_497,Top_498,Top_499,Top_500,Top_501,Top_502,Top_503,Top_504,Top_505,Top_506,Top_507,Top_508,Top_509,Top_510,Top_511,Top_512,Top_5

In [106]:
top_df.shape

(5435, 2048)

In [107]:
filename = 'Output/Topological_BMatrix_map_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
top_df.to_csv(filename, sep='\t', compression='gzip')

## Topological Torsion Fingerprints

In [108]:
topt_fps = [rdMolDescriptors.GetHashedTopologicalTorsionFingerprintAsBitVect(x) for x in df['Molecule']]

topt_np_fps = []
for fp in topt_fps:
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp, arr)
    topt_np_fps.append(arr)

topt_df = pd.DataFrame(topt_np_fps, index = all_drugs)

column_labels = []
for col in topt_df.columns:
    column_labels.append('Topt_' + str(col))
topt_df.columns = column_labels

filename = 'Output/TopologicalTorsion_BMatrix_map_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
topt_df.to_csv(filename, sep='\t', compression='gzip')

## RDK Fingerprint

In [111]:
#Change maxPath as needed
rdkf_fps = [Chem.RDKFingerprint(x,maxPath=2) for x in df['Molecule']]

rdkf_np_fps = []
for fp in rdkf_fps:
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp, arr)
    rdkf_np_fps.append(arr)

rdkf_df = pd.DataFrame(rdkf_np_fps, index = all_drugs)

column_labels = []
for col in rdkf_df.columns:
    column_labels.append('RDKf2_' + str(col))
rdkf_df.columns = column_labels

filename = 'Output/RDKfps2_BMatrix_map_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
rdkf_df.to_csv(filename, sep='\t', compression='gzip')